In [1]:
pip install librosa

  Using cached librosa-0.11.0-py3-none-any.whl.metadata (8.7 kB)
  Using cached audioread-3.0.1-py3-none-any.whl.metadata (8.4 kB)
  Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl.metadata (16 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached soxr-0.5.0.post1-cp312-abi3-win_amd64.whl.metadata (5.6 kB)
Using cached librosa-0.11.0-py3-none-any.whl (260 kB)
Using cached audioread-3.0.1-py3-none-any.whl (23 kB)
Using cached pooch-1.8.2-py3-none-any.whl (64 kB)
Using cached soundfile-0.13.1-py2.py3-none-win_amd64.whl (1.0 MB)
Using cached soxr-0.5.0.post1-cp312-abi3-win_amd64.whl (164 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import librosa
import numpy as np
from PIL import Image
from tqdm import tqdm

# 경로 설정
AUDIO_ROOT = './train_audio'
OUTPUT_ROOT = './train_audio_mel'

# 멜 스펙트로그램 저장 함수
def save_mel_spectrogram(audio_path, save_path, sr=32000, n_mels=128, fmax=16000):
    try:
        y, sr = librosa.load(audio_path, sr=sr)
        mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, fmax=fmax)
        mel_db = librosa.power_to_db(mel, ref=np.max)

        mel_db -= mel_db.min()
        mel_db /= mel_db.max()
        mel_img = (mel_db * 255).astype(np.uint8)
        mel_rgb = np.stack([mel_img]*3, axis=-1)

        image = Image.fromarray(mel_rgb)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        image.save(save_path)
        return True
    except Exception as e:
        print(f"❌ 변환 실패: {audio_path}, 이유: {e}")
        return False

# 전체 오디오 폴더를 돌면서 변환 진행
def process_all_audio():
    for species in tqdm(os.listdir(AUDIO_ROOT), desc='🦜 종별 폴더'):
        species_path = os.path.join(AUDIO_ROOT, species)
        if not os.path.isdir(species_path):
            continue

        output_species_path = os.path.join(OUTPUT_ROOT, species)
        os.makedirs(output_species_path, exist_ok=True)

        for file in os.listdir(species_path):
            if file.endswith('.ogg') or file.endswith('.wav'):
                audio_path = os.path.join(species_path, file)
                save_path = os.path.join(output_species_path, file.rsplit('.', 1)[0] + '.png')
                save_mel_spectrogram(audio_path, save_path)

if __name__ == "__main__":
    process_all_audio()
    print("✅ 멜 스펙트로그램 생성 완료!")


🦜 종별 폴더: 100%|██████████| 206/206 [00:02<00:00, 102.20it/s]

✅ 멜 스펙트로그램 생성 완료!
